In [ ]:
%load_ext autoreload
%autoreload 2
import sys, os
import shutil
import tarfile
import errno
from datetime import datetime
import numpy as np
import scipy.optimize as optimize
import scipy.fftpack as fft
import scipy.linalg as linalg
from scipy.stats import linregress

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from BandPass2 import *
from fitFunctions import *
from partialfitting import *
from tqdm import *
from fitphases import *
from fitGammaRatio import *
from chi2 import *
from corrphase import *
from corr2Pi import *

mpl.rcParams['figure.figsize'] = [8.0, 6.0]
mpl.rcParams['figure.dpi'] = 80
mpl.rcParams['savefig.dpi'] = 100
mpl.rcParams['font.size'] = 15

In [ ]:
dirName = "/home/jean/RomalisResearch/Data/18-12-15/" # Directory with data
files = [f for f in os.listdir(dirName) if f.endswith('.txt')]
gHe = 20378.9
gNe = 2*np.pi*336.1
gRatio = gHe/gNe

ks1 = 6 # First file index
ks2 = 49 # Last file index
n_ave = 1 # Number of points to average over

dt = 1e-3*n_ave
Fs = 1/dt
D1Start = 20 # Start of first detection (currently only uses this)
D1EndTimes = [500] # End of first detection (currently only uses this)
D2Start = 200 # Start of second detection (currently doesn't use this)
D2EndTimes = [240] # End of second detection (currently doesn't use this)
absSigma = True

delta_t = 2/1.56 # Time length of a subsection

In [ ]:
saveDirName = os.path.join(
    '/home/jean/RomalisResearch/pulsed_fitter_current/Results/',
    datetime.now().strftime('%Y-%m-%d_%H-%M'))
os.makedirs(saveDirName)
for k_ind in range(ks1, ks2+1):
    print(k_ind)
    k = k_ind - ks1 # file number
    print(files[k_ind])
    initialfitting(saveDirName, k_ind, k, n_ave, dirName, files, dt, Fs, D1Start, D1EndTimes, D2Start, D2EndTimes, delta_t)
    correctPhases(saveDirName, k_ind)
    fitPhases(saveDirName, k_ind, absSigma)

In [ ]:
for k_ind in range(ks1, ks2+1):
    k = k_ind - ks1 # file number
    corrphase(saveDirName, k_ind, absSigma)
with tarfile.open(saveDirName+".gz", "w:gz") as tar_handle:
    for root, dirs, file in os.walk(saveDirName):
        for f in file:
            tar_handle.add(os.path.join(root, f), arcname=f)
shutil.rmtree(saveDirName)